# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_weather = pd.read_csv("../Output/weather_check.csv")
city_weather.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Ushuaia,-54.80,-68.30,37.11,80,52,8.66,AR,1603109309
1,1,New Norfolk,-42.78,147.06,51.01,73,81,1.01,AU,1603109310
2,2,Doha,25.29,51.53,95.00,33,0,10.00,QA,1603109016
3,3,Marsh Harbour,26.54,-77.06,80.83,76,98,19.89,BS,1603109313
4,4,Sola,-13.88,167.55,79.90,79,39,17.74,VU,1603109314


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
locations = city_weather[["Lat", "Lng"]]
humidity = city_weather["Humidity"]
max_humidity = humidity.max()

heat_map_figure = gmaps.figure(layout = {
        'width': '1200px',
        'height': '800px'})
heat_map_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = max_humidity, 
                                    point_radius = 3)

heat_map_figure.add_layer(heat_map_layer)

heat_map_figure

Figure(layout=FigureLayout(height='800px', width='1200px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrow_down_cities_df = city_weather.loc[(city_weather["Max Temp"].astype(float) >= 70.0) & 
                                         (city_weather["Max Temp"].astype(float) <= 80.0) & 
                                         (city_weather["Cloudiness"].astype(float) == 0.0) & 
                                         (city_weather["Wind Speed"].astype(float) < 10.0)]

narrow_down_cities_df = narrow_down_cities_df.dropna(how = "any")

del narrow_down_cities_df["Unnamed: 0"]

narrow_down_cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
50,Bāfq,31.61,55.41,79.57,10,0,9.01,IR,1603109366
70,Santa Maria,-29.68,-53.81,75.40,61,0,5.08,BR,1603109394
180,Faya,18.39,42.45,73.71,18,0,6.58,SA,1603109534
193,Qazvin,36.28,50.00,72.36,17,0,3.38,IR,1603109268
340,Ardakān,32.31,54.02,79.92,11,0,8.34,IR,1603109747
355,Doaba,33.42,70.74,79.93,15,0,5.88,PK,1603109765
361,Yavaros,26.70,-109.52,72.00,86,0,1.99,MX,1603109773
368,Badiraguato,25.37,-107.52,79.00,92,0,2.84,MX,1603109783
399,Yangi Marg`ilon,40.43,71.72,71.15,21,0,1.61,UZ,1603109824
444,Chardara,41.25,67.97,70.20,25,0,4.25,KZ,1603109882


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = pd.DataFrame(narrow_down_cities_df, columns=["City", "Country", "Lat", "Lng"])
hotels = []

for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    params = {
        "location" : f"{latitude},{longitude}", 
        "radius" : 5000, 
        "types" : "hotel", 
        "key" : g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response =  requests.get(base_url, params = params)
    
    response_json = response.json()
    
    try:
        hotels.append(response_json['results'][0]['name'])
    except IndexError:
        hotels.append(np.nan)
    
    time.sleep(1)

In [8]:
hotel_df["Hotel Name"] = hotels
hotel_df

,City,Country,Lat,Lng,Hotel Name
50,Bāfq,IR,31.61,55.41,Bafgh
70,Santa Maria,BR,-29.68,-53.81,Santa Maria
180,Faya,SA,18.39,42.45,الغال
193,Qazvin,IR,36.28,50.00,Qazvin
340,Ardakān,IR,32.31,54.02,Meybod
355,Doaba,PK,33.42,70.74,Sarozai
361,Yavaros,MX,26.70,-109.52,Yavaros
368,Badiraguato,MX,25.37,-107.52,Badiraguato
399,Yangi Marg`ilon,UZ,40.43,71.72,Fergana
444,Chardara,KZ,41.25,67.97,Shardara


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

heat_map_figure.add_layer(markers)

# Display figure
heat_map_figure

Figure(layout=FigureLayout(height='800px', width='1200px'))